In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../'))

In [2]:
import pandas as pd

from mcda.electre.concordance import concordance_reinforced, concordance_reinforced_comprehensive, concordance_comprehensive
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.core.functions import Threshold

1. Create dataframes

In [3]:
alternative_names = ['Spain', 'France' , 'Italy' , 'Greece' , 'Portugal', 'United Kingdom',
                     'Norway', 'Brazil', 'Japan', 'Bahamas', 'Canada', 'Switzerland', 'Australia',
                     'New Zealand', 'Mexico', 'Germany', 'Thailand', 'United States']
criteria_names = ['HDI', 'Crime rate', 'Climate', 'Wealth Gini']

In [4]:
alternatives_perform = [[0.905, 33, 5, 69],
                        [0.903, 51, 4, 70],
                        [0.895, 44, 5, 67],
                        [0.887, 45, 5, 68],
                        [0.886, 29, 4, 70],
                        [0.929, 46, 3, 70],
                        [0.961, 33, 2, 79],
                        [0.754, 67, 4, 89],
                        [0.925, 22, 4, 64],
                        [0.812, 62, 5, 89],
                        [0.936, 41, 3, 72],
                        [0.962, 21, 3, 77],
                        [0.951, 43, 5, 66],
                        [0.937, 42, 4, 70],
                        [0.758, 54, 5, 80],
                        [0.942, 35, 3, 78],
                        [0.8, 39, 4, 77],
                        [0.921, 47, 4, 85]
                        ]
countries = pd.DataFrame(alternatives_perform,
    index=alternative_names,
    columns=criteria_names,
)
countries


,HDI,Crime rate,Climate,Wealth Gini
Spain,0.905,33,5,69
France,0.903,51,4,70
Italy,0.895,44,5,67
Greece,0.887,45,5,68
Portugal,0.886,29,4,70
United Kingdom,0.929,46,3,70
Norway,0.961,33,2,79
Brazil,0.754,67,4,89
Japan,0.925,22,4,64
Bahamas,0.812,62,5,89


2. Create series with thresholds and specify criteria scales and weights

In [5]:
scales = pd.Series([
    QuantitativeScale(0.7, 1.0),
    QuantitativeScale(15, 75, PreferenceDirection.MIN),
    QuantitativeScale(1, 5),
    QuantitativeScale(50, 100, PreferenceDirection.MIN)
], index=criteria_names)
scales

HDI            Scale [0.7, 1.0], max direction
Crime rate       Scale [15, 75], min direction
Climate            Scale [1, 5], max direction
Wealth Gini     Scale [50, 100], min direction
dtype: object

In [6]:
weights = pd.Series([0.3, 0.25, 0.35, 0.1], index=criteria_names)
weights

HDI            0.30
Crime rate     0.25
Climate        0.35
Wealth Gini    0.10
dtype: float64

In [7]:
indifference = pd.Series([
    Threshold(0, 0.01),
    Threshold(0, 4),
    Threshold(0, 0),
    Threshold(0, 4),
], index=criteria_names)
indifference

HDI            Threshold: alpha=0, beta=0.01
Crime rate        Threshold: alpha=0, beta=4
Climate           Threshold: alpha=0, beta=0
Wealth Gini       Threshold: alpha=0, beta=4
dtype: object

In [8]:
preference = pd.Series([
    Threshold(0.01, 0.02),
    Threshold(0, 8),
    Threshold(0, 1),
    Threshold(0, 7)
], index=criteria_names)
preference

HDI            Threshold: alpha=0.01, beta=0.02
Crime rate           Threshold: alpha=0, beta=8
Climate              Threshold: alpha=0, beta=1
Wealth Gini          Threshold: alpha=0, beta=7
dtype: object

In [9]:
reinforce = pd.Series([
    Threshold(0, 0.1),
    Threshold(0, 20),
    None,
    None
], index=criteria_names)
reinforce

HDI            Threshold: alpha=0, beta=0.1
Crime rate      Threshold: alpha=0, beta=20
Climate                                None
Wealth Gini                            None
dtype: object

In [10]:
reinforced_factors = pd.Series([ 
    1.4,
    1.2,
    None,
    None
], index=criteria_names)
reinforced_factors


HDI            1.4
Crime rate     1.2
Climate        NaN
Wealth Gini    NaN
dtype: float64

3. Let's begin with comprehensive concordance calculations

In [11]:
concordance_reinforced_comprehensive(
    countries.loc['France'],
    countries.loc['Spain'],
    scales,
    weights,
    indifference,
    preference,
    reinforce,
    reinforced_factors
)

0.4

In [12]:
concordance = concordance_reinforced(
    countries,
    scales,
    weights,
    indifference,
    preference,
    reinforce,
    reinforced_factors
)
print(concordance)

                   Spain    France     Italy    Greece  Portugal  \
Spain           1.000000  1.000000  1.000000  1.000000     1.000   
France          0.400000  1.000000  0.462500  0.525000     0.750   
Italy           0.750000  1.000000  1.000000  1.000000     0.750   
Greece          0.622814  0.904610  1.000000  1.000000     0.750   
Portugal        0.506840  0.893955  0.650000  0.650000     1.000   
United Kingdom  0.400000  0.650000  0.650000  0.650000     0.400   
Norway          0.550000  0.550000  0.550000  0.550000     0.550   
Brazil          0.000000  0.350000  0.000000  0.000000     0.350   
Japan           0.650000  1.000000  0.666667  0.666667     1.000   
Bahamas         0.350000  0.350000  0.350000  0.350000     0.350   
Canada          0.400000  0.650000  0.616667  0.650000     0.400   
Switzerland     0.550000  0.571429  0.571429  0.571429     0.550   
Australia       0.750000  1.000000  1.000000  1.000000     0.750   
New Zealand     0.400000  1.000000  0.650000  0.

Let's define profiles.

In [13]:
alternative_names_profiles_out = ['Spain', 'France' , 'Italy' , 'Portugal', 'United Kingdom',
                     'Norway', 'Brazil', 'Japan', 'Bahamas', 'Canada', 'Switzerland', 'Australia',
                     'New Zealand', 'Mexico', 'Germany']

In [14]:
alternatives_perform_po = [[0.905, 33, 5, 69],
                        [0.903, 51, 4, 70],
                        [0.895, 44, 5, 67],
                        [0.886, 29, 4, 70],
                        [0.929, 46, 3, 70],
                        [0.961, 33, 2, 79],
                        [0.754, 67, 4, 89],
                        [0.925, 22, 4, 64],
                        [0.812, 62, 5, 89],
                        [0.936, 41, 3, 72],
                        [0.962, 21, 3, 77],
                        [0.951, 43, 5, 66],
                        [0.937, 42, 4, 70],
                        [0.758, 54, 5, 80],
                        [0.942, 35, 3, 78]
                        ]
countries_profiles_out = pd.DataFrame(alternatives_perform_po,
    index=alternative_names_profiles_out,
    columns=criteria_names,
)
Greece = [0.887, 45, 5, 68]
Thailand = [0.8, 39, 4, 77]
USA = [0.921, 47, 4, 85]
profile_names = ["United States", "Thailand", "Greece"]
profiles = pd.DataFrame(
    [USA, Thailand, Greece],
    index=profile_names,
    columns=criteria_names
) 
profiles

,HDI,Crime rate,Climate,Wealth Gini
United States,0.921,47,4,85
Thailand,0.800,39,4,77
Greece,0.887,45,5,68


In [15]:
concordance_profiles = concordance_reinforced(
    countries_profiles_out,
    scales,
    weights,
    indifference,
    preference,
    reinforce,
    reinforced_factors,
    profiles
)
concordance_profiles

(                United States  Thailand    Greece
 Spain                0.905512  1.000000  1.000000
 France               0.873883  0.776786  0.525000
 Italy                0.746702  0.937500  1.000000
 Portugal             0.700000  1.000000  0.650000
 United Kingdom       0.650000  0.520089  0.650000
 Norway               0.650000  0.687500  0.550000
 Brazil               0.450000  0.350000  0.000000
 Japan                1.000000  1.000000  0.666667
 Bahamas              0.450000  0.650000  0.350000
 Canada               0.650000  0.687500  0.650000
 Switzerland          0.666667  0.687500  0.571429
 Australia            1.000000  1.000000  1.000000
 New Zealand          1.000000  1.000000  0.650000
 Mexico               0.512500  0.450000  0.350000
 Germany              0.650000  0.687500  0.550000,
                   Spain   France  Italy  Portugal  United Kingdom    Norway  \
 United States  0.300000  0.90000   0.55      0.65             0.9  0.383333   
 Thailand       0.12500

It's time for defining veto and calculating discordance indices.

In [16]:
from mcda.electre.discordance import discordance_marginals, discordance_comprehensive

In [17]:
veto = pd.Series([
    None,
    None,
    Threshold(2, 5),
    None
], index=criteria_names)
veto

HDI                                  None
Crime rate                           None
Climate        Threshold: alpha=2, beta=5
Wealth Gini                          None
dtype: object

In [18]:
discordance_comprehensive(
    countries.loc['Spain'],
    countries.loc['Norway'],
    scales,
    weights,
    preference,
    veto
)

0.0

In [19]:
discord = discordance_marginals(countries,
                scales, 
                preference,
                veto)
discord_prof = discordance_marginals(countries_profiles_out,
                scales, 
                preference,
                veto, profiles_perform=profiles)

Next step is to calculate non-discordance and pass non-discordance and concordance matrices to credibility module.

In [20]:
from mcda.electre.discordance import non_discordance, NonDiscordanceType
from mcda.electre.credibility import credibility_comprehensive

In [21]:
non_discordance_d = non_discordance(discord, NonDiscordanceType.D, concordance)
non_discordance_d_prof = (non_discordance(discord_prof[0], NonDiscordanceType.D, concordance_profiles[0]), non_discordance(discord_prof[1], NonDiscordanceType.D, concordance_profiles[1]))
non_discordance_d[non_discordance_d != 1]

,Spain,France,Italy,Greece,Portugal,United Kingdom,Norway,Brazil,Japan,Bahamas,Canada,Switzerland,Australia,New Zealand,Mexico,Germany,Thailand,United States
Spain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Italy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Greece,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Portugal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
United Kingdom,0.90,NaN,0.90,0.90,NaN,NaN,NaN,NaN,NaN,0.90,NaN,NaN,0.90,NaN,0.90,NaN,NaN,NaN
Norway,0.75,0.875,0.75,0.75,0.875,NaN,NaN,0.875,0.875,0.75,NaN,NaN,0.75,0.875,0.75,NaN,0.875,0.875
Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bahamas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
credibility = credibility_comprehensive(concordance, non_discordance_d)
credibility_prof = (credibility_comprehensive(concordance_profiles[0], non_discordance_d_prof[0]), 
               credibility_comprehensive(concordance_profiles[1], non_discordance_d_prof[1]))
credibility


,Spain,France,Italy,Greece,Portugal,United Kingdom,Norway,Brazil,Japan,Bahamas,Canada,Switzerland,Australia,New Zealand,Mexico,Germany,Thailand,United States
Spain,1.000000,1.000000,1.000000,1.000000,1.00000,0.779528,0.700000,1.000000,0.559186,1.000000,0.700000,0.450000,0.700000,0.700000,1.000000,0.700000,1.000000,0.905512
France,0.400000,1.000000,0.462500,0.525000,0.75000,0.685267,0.450000,1.000000,0.494158,0.650000,0.450000,0.450000,0.100000,0.450000,0.687500,0.450000,0.776786,0.873883
Italy,0.750000,1.000000,1.000000,1.000000,0.75000,0.700000,0.450000,1.000000,0.450000,1.000000,0.700000,0.450000,0.700000,0.700000,1.000000,0.450000,0.937500,0.746702
Greece,0.622814,0.904610,1.000000,1.000000,0.75000,0.700000,0.450000,1.000000,0.450000,1.000000,0.700000,0.450000,0.700000,0.700000,1.000000,0.450000,0.875000,0.700000
Portugal,0.506840,0.893955,0.650000,0.650000,1.00000,0.700000,0.700000,1.000000,0.445833,0.666667,0.700000,0.450000,0.350000,0.700000,0.700855,0.700000,1.000000,0.700000
United Kingdom,0.360000,0.650000,0.585000,0.585000,0.40000,1.000000,0.450000,0.700855,0.333333,0.618750,0.937500,0.450000,0.417037,0.650000,0.618750,0.703344,0.520089,0.650000
Norway,0.412500,0.481250,0.412500,0.412500,0.48125,0.550000,1.000000,0.613248,0.262500,0.525641,0.550000,0.400000,0.412500,0.481250,0.525641,0.650000,0.601563,0.568750
Brazil,0.000000,0.350000,0.000000,0.000000,0.35000,0.350000,0.350000,1.000000,0.350000,0.287500,0.350000,0.350000,0.000000,0.350000,0.300000,0.350000,0.350000,0.450000
Japan,0.650000,1.000000,0.666667,0.666667,1.00000,1.000000,0.700000,1.000000,1.000000,0.700855,0.984416,0.700000,0.429190,0.968831,0.700855,0.890909,1.000000,1.000000
Bahamas,0.350000,0.350000,0.350000,0.350000,0.35000,0.350000,0.350000,1.000000,0.350000,1.000000,0.350000,0.350000,0.350000,0.350000,0.650000,0.350000,0.650000,0.450000


In [23]:
from mcda.electre.outranking.sorting import assign_tri_b_class
from mcda.electre.outranking.crisp_outranking import crisp_cut

In [24]:
crisp_table = (crisp_cut(credibility_prof[0], 0.7), crisp_cut(credibility_prof[1], 0.7))

In [25]:
crisp_table[0]

,United States,Thailand,Greece
Spain,True,True,True
France,True,True,False
Italy,True,True,True
Portugal,True,True,False
United Kingdom,False,False,False
Norway,False,False,False
Brazil,False,False,False
Japan,True,True,False
Bahamas,False,False,False
Canada,False,False,False


In [26]:
crisp_table[1]

,Spain,France,Italy,Portugal,United Kingdom,Norway,Brazil,Japan,Bahamas,Canada,Switzerland,Australia,New Zealand,Mexico,Germany
United States,False,True,False,False,True,False,True,False,False,False,False,False,True,False,False
Thailand,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True
Greece,False,True,True,True,True,False,True,False,True,True,False,True,True,True,False


In [27]:
classes_tri_b = pd.Series(["Thailand", "United States", "Greece", None],
                                index=["Not really", "Might consider", "Better than USA", "Paradise"])

In [28]:
assign_tri_b_class(crisp_table[0], crisp_table[1], classes_tri_b)


Spain                    (Better than USA, Paradise)
France            (Better than USA, Better than USA)
Italy                    (Better than USA, Paradise)
Portugal          (Better than USA, Better than USA)
United Kingdom          (Not really, Might consider)
Norway                        (Not really, Paradise)
Brazil                      (Not really, Not really)
Japan                    (Better than USA, Paradise)
Bahamas                (Not really, Better than USA)
Canada                 (Not really, Better than USA)
Switzerland                   (Not really, Paradise)
Australia                (Better than USA, Paradise)
New Zealand       (Better than USA, Better than USA)
Mexico                 (Not really, Better than USA)
Germany                     (Not really, Not really)
dtype: object